In [47]:
import pandas as pd
from sklearn.metrics import ndcg_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy.stats import mode

train_df = pd.read_csv('train/train_df.csv')
test_df = pd.read_csv('test/test_df.csv')
print("ok")

ok


In [48]:
X_train = train_df.drop(['search_id', 'target'], axis=1)
y_train = train_df['target']
X_test = test_df.drop(['search_id', 'target'], axis=1)
y_test = test_df['target']
print("ok")

ok


In [49]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, average_precision_score
import pandas as pd

def evaluate(model, testing_set_x, testing_set_y):
    # Получение предсказанных вероятностей для положительного класса
    predictions_proba = model.predict(testing_set_x)
    
    # Вычисление метрик
    accuracy = accuracy_score(testing_set_y, predictions_proba >= 0.5)
    roc_auc = roc_auc_score(testing_set_y, predictions_proba)
    precision = precision_score(testing_set_y, predictions_proba >= 0.5)
    recall = recall_score(testing_set_y, predictions_proba >= 0.5)
    pr_auc = average_precision_score(testing_set_y, predictions_proba)
    
    # Формирование результата
    result = pd.DataFrame([[accuracy, precision, recall, roc_auc, pr_auc]], columns=['Accuracy', 'Precision', 'Recall', 'ROC_auc', 'PR_auc'])
    return result

In [51]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)


# Предполагается, что X и y уже определены в вашем наборе данных
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание ансамбля из 10 случайных лесов
ensemble = [RandomForestClassifier(n_estimators=100, random_state=42+i) for i in range(10)]

# Обучение каждой модели-члена на подмножестве данных
for i, model in enumerate(ensemble):
    # Генерация индексов подмножества с заменой
    sample_indices = np.random.choice(len(X_train), size=len(X_train), replace=True)
    X_subset, y_subset = X_train[sample_indices], y_train[sample_indices]
    model.fit(X_subset, y_subset)
    print(f"Модель {i+1} обучена.")

# Совместное предсказание всех моделей ансамбля
predictions = np.array([model.predict(X_test) for model in ensemble])

# Использование моды для определения окончательных предсказаний ансамбля
ensemble_predictions = mode(predictions, axis=0)[0][0]

# Оценка точности ансамбля
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Точность ансамбля: {ensemble_accuracy}")
print(evals_result)


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[17]	valid_0's binary_logloss: 0.305711	valid_0's auc: 0.725428
{'valid_0': OrderedDict([('binary_logloss', [0.6455847351171102, 0.6067440196583501, 0.5695335677496439, 0.5391916555692454, 0.5128062948576435, 0.4859830400044182, 0.4609022804831424, 0.4439248933006247, 0.42810114210546807, 0.4062344196969009, 0.390494954289057, 0.3745143461192054, 0.3575094401146548, 0.3418940980426408, 0.3305702348530008, 0.31706559872354695, 0.3057112719653844, 0.29266489457428607, 0.28217412049342017, 0.2738487731538682, 0.26874582096111027, 0.2604089153935214, 0.2519222113548596, 0.2443139000433738, 0.23739611022771712, 0.2313598240168327, 0.22672161352919237]), ('auc', [0.5156993901239425, 0.5643320873499902, 0.5914814086169585, 0.663623844186504, 0.6697816250245918, 0.6653452685421994, 0.685638402518198, 0.6865040330513477, 0.6896419437340153, 0.690537084398977, 0.6954456029903601, 0.7014361597481802, 0

In [52]:
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)


In [53]:
# Мы могли бы группировать данные по search_id и рассчитывать NDCG для каждой группы,
# но поскольку задача стоит рассчитать NDCG для всех документов, мы рассчитаем её глобально.
ndcg_score = ndcg_score([y_test], [y_pred], k=None)
print(f'NDCG Score: {ndcg_score}')


NDCG Score: 0.4875445357641325
